# Assignment 7: object-oriented programming
- **you will learn:** How to use subclassing when defining models
- **task:**  See section 7.3 below
- **deadline:** 12.01.2026
- [classes in python](https://docs.python.org/3/tutorial/classes.html)
- 📝 **Reminder:** Sync your GitHub repository with the main course repository, update your project in PyCharm, and after completing the assignment, commit and push your changes back to GitHub.
---

In [23]:
import numpy as np
import sklearn as sk

print("NumPy version:", np.__version__)
print("Scikit learn version:", sk.__version__)

NumPy version: 2.3.2
Scikit learn version: 1.7.1


## **7.1 Subclassing a Base Model**

- In machine learning libraries such as **scikit-learn**, predictive models (for example,  
  [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html))  
  are implemented as classes that **inherit from shared base classes**.
- These base classes define a **common interface and internal structure**, ensuring that all models expose a consistent set of methods, most notably `.fit()` for model training and `.predict()` for inference.
- This standardized object-oriented design enables **model interchangeability**: different algorithms can be substituted with minimal code changes, often requiring the modification of only a single line.
- Such a unified framework greatly improves **maintainability, extensibility, and reproducibility**, which are essential properties for large-scale data science and machine learning workflows.
- Further technical details can be found in the  
  [scikit-learn developer documentation](https://scikit-learn.org/stable/developers/develop.html).

In [24]:
# --------------------------------------------------
# Generate synthetic regression data
# --------------------------------------------------
np.random.seed(42)

X = np.linspace(0, 10, 100).reshape(-1, 1)   # feature matrix
y = 3.0 + 2.5 * X.squeeze() + np.random.normal(0, 2, size=X.shape[0])  # target
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(
    X, y, test_size=0.25, random_state=42
)

# --------------------------------------------------
# Create, fit, and evaluate model
# --------------------------------------------------
# NOTE: To use a different model (e.g. Ridge, Lasso, RandomForest),
# only the following line typically needs to be changed.
model = sk.linear_model.LinearRegression()

model.fit(X_train, y_train)

# Predict on unseen test data
y_pred = model.predict(X_test)

# Inspect first predictions
y_pred[:5]

array([23.8253613 , 16.17800946, 20.51150884, 14.13871564, 13.88380391])

- Similarly, the same standardized method structure is used for **data transformers** (e.g. scalers, encoders), which also expose a consistent interface.
- In the following sections, we demonstrate how to **define custom models using subclassing**, following the same design principles.
- This approach not only enables the creation of customized models by extending or modifying existing implementations, but it also represents the **primary programming paradigm for neural networks**, as will be illustrated in later sections.

In [25]:
# ---------------------------------------------------------------------
# Base model definition
# ---------------------------------------------------------------------
# Our models will implement the methods 'fit', 'predict', and 'summary'.
# These methods define a common interface and must be overridden by
# every concrete subclass.
class BaseModel:
    """
    Base class for all models.

    This class defines a shared interface that all models must follow.
    It is not intended to be used directly.
    """

    def __init__(self):
        self.is_fitted = False

    def fit(self, X, y):
        """Train the model."""
        raise NotImplementedError("fit() must be implemented by subclasses")

    def predict(self, X):
        """Generate predictions for input data."""
        raise NotImplementedError("predict() must be implemented by subclasses")

    def summary(self):
        """Display a summary of the trained model."""
        raise NotImplementedError("summary() must be implemented by subclasses")

In [26]:
# ---------------------------------------------------------------------
# Simple baseline model
# ---------------------------------------------------------------------
class SimpleAverageModel(BaseModel):
    """
    Baseline model that always predicts the average of the training target.

    This model completely ignores the input features and is often used
    as a simple performance baseline.
    """

    def __init__(self):
        super().__init__()
        self.mean_ = None

    def fit(self, X, y):
        y = np.asarray(y)
        self.mean_ = y.mean()
        self.is_fitted = True

    def predict(self, X):
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before prediction")

        # Predict the same value for all inputs
        return np.full(shape=len(X), fill_value=self.mean_)

    def summary(self):
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before summary")

        print("Simple Average Model Summary")
        print("----------------------------")
        print(f"Predicted constant value (mean): {self.mean_:.4f}")

In [27]:
# ---------------------------------------------------------------------
# Linear regression model
# ---------------------------------------------------------------------
class LinearRegressionModel(BaseModel):
    """
    Simple linear regression using the closed-form least squares solution.
    """

    def __init__(self):
        super().__init__()   # initialize BaseModel
        self.beta_0 = None
        self.beta_1 = None

    def fit(self, X, y):
        """
        Train the model using ordinary least squares.
        """
        X = np.asarray(X)
        y = np.asarray(y)

        x_mean = X.mean()
        y_mean = y.mean()

        self.beta_1 = np.sum((X - x_mean) * (y - y_mean)) / np.sum((X - x_mean) ** 2)
        self.beta_0 = y_mean - self.beta_1 * x_mean

        self.is_fitted = True

    def predict(self, X):
        """
        Predict values using the trained linear model.
        """
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before prediction")

        X = np.asarray(X)
        return self.beta_0 + self.beta_1 * X

    def summary(self):
        """
        Print model parameters.
        """
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before summary")

        print("Linear Regression Model Summary")
        print("-------------------------------")
        print(f"Intercept (beta_0): {self.beta_0:.4f}")
        print(f"Slope     (beta_1): {self.beta_1:.4f}")

In [28]:
# Generate synthetic data
np.random.seed(42)
X = np.linspace(0, 10, 50)
y = 3.0 + 2.5 * X + np.random.normal(0, 2, size=len(X))

# Create and train model
model = LinearRegressionModel()  # you can replace with SimpleAverageModel()
model.fit(X, y)

# Inspect parameters
model.summary()

Linear Regression Model Summary
-------------------------------
Intercept (beta_0): 3.1289
Slope     (beta_1): 2.3840


In [29]:
# ---------------------------------------------------------------------
# Linear regression model in improved custom version
# ---------------------------------------------------------------------
class LinearRegressionWithStats(LinearRegressionModel):
    """
    Linear regression with goodness-of-fit statistics.
    """

    def r_squared(self, X, y):
        """
        Compute R² score.
        """
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted first")

        y = np.asarray(y)
        y_pred = self.predict(X)

        ss_total = np.sum((y - y.mean()) ** 2)
        ss_residual = np.sum((y - y_pred) ** 2)

        return 1 - ss_residual / ss_total

    def residuals(self, X, y):
        """
        Return residuals y - y_hat.
        """
        return y - self.predict(X)

In [30]:
model_stats = LinearRegressionWithStats()
model_stats.fit(X, y)

model_stats.summary()

r2 = model_stats.r_squared(X, y)
print(f"R^2 score: {r2:.4f}")

Linear Regression Model Summary
-------------------------------
Intercept (beta_0): 3.1289
Slope     (beta_1): 2.3840
R^2 score: 0.9372


In [31]:
# Inspect residuals
res = model_stats.residuals(X, y)
res[:5]

array([ 0.86454212, -0.38174809,  1.21382428,  2.98817361, -0.50252616])

## 7.2 Subclassing for Neural Networks

In PyTorch, creating custom neural network architectures is typically done by **subclassing `torch.nn.Module`**. This approach provides full control over the network layers, forward propagation, and any additional operations you might want to include.  

### Why Subclass `nn.Module`?

- Allows defining **custom architectures** beyond pre-built models.  
- Provides a clear separation between **layers** and **forward computation**.  
- Integrates with PyTorch’s **autograd** for automatic differentiation, details are provided in **Data Science 2** course.  

### Example: Two-Layer Perceptron (MLP)

Below is an example of a **two-layer multilayer perceptron (MLP)** implemented for regression tasks. For further reference on MLPs, see [this tutorial](https://www.geeksforgeeks.org/deep-learning/multi-layer-perceptron-learning-in-tensorflow/).  

```python
import torch

class TwoLayerPerceptron(torch.nn.Module):
    """
    Two-layer multilayer perceptron (MLP) for regression tasks.

    Architecture:
        Input -> Linear -> Tanh -> Linear -> Output
    """

    def __init__(self, input_dim=1, hidden_dim=10, output_dim=1):
        """
        Initialize the network layers.

        Args:
            input_dim (int): Number of input features.
            hidden_dim (int): Number of neurons in the hidden layer.
            output_dim (int): Number of output neurons.
        """
        super().__init__()
        self.hidden = torch.nn.Linear(input_dim, hidden_dim)
        self.activation = torch.nn.Tanh()
        self.output = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """
        Define the forward pass of the network.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, input_dim)

        Returns:
            torch.Tensor: Output predictions
        """
        x = self.hidden(x)
        x = self.activation(x)
        x = self.output(x)
        return x


## 7.3 Homework: Subclassing in Data Science

### Task Overview
In this assignment, you will practice **subclassing** by extending a linear regression model. Your goal is to enhance the standard model with an option to apply a **logarithmic transformation** to the response variable (`y`).

---

### Your Task
1. **Create a subclass** called `LinearRegressionModelWithTransformation` that inherits from `LinearRegressionModel`.

2. **Constructor (`__init__`)**  
   - Add a boolean parameter `use_log` to the constructor, defaulting to `False`.

3. **Override the `fit` method**  
   - If `use_log` is `True`, apply `np.log` to the target variable `y` before fitting.
   - If any `y` values are non-positive, shift all values so they are positive and print a message indicating the shift.
   - If `use_log` is `False`, fit the model normally.
   - **Important:** Do not re-implement the computation of `beta_0` and `beta_1`. Instead, transform the data as needed and call the parent `.fit()` method using `super().fit(X, y_transformed)`.

4. **Override the `predict` method**  
   - Ensure predictions are consistent with the `use_log` parameter (i.e. if `use_log=True`, return exponentiated predictions so that outputs are on the original scale.)

5. **`summary` method**  
   - The `summary` method remains unchanged

---

### Example Usage

```python
import numpy as np

X = np.array([1, 2, 3, 4, 5])
y = np.array([2.7, 7.4, 20.1, 54.6, 148.4])  # exponential trend

model_log = LinearRegressionModelWithTransformation(use_log=True)
model_log.fit(X, y)
print("Log-transformed predictions:", model_log.predict([6, 7]))
model_log.summary()

model_plain = LinearRegressionModelWithTransformation(use_log=False)
model_plain.fit(X, y)
print("Normal predictions:", model_plain.predict([6, 7]))
model_plain.summary()
